In [ ]:
% run 0-utils.ipynb

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import torch.utils.data as data_utils
import torchvision.datasets as dset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from tensorboardX import SummaryWriter
import networkx as nx
from scipy import spatial
import scipy.sparse.csgraph
from timeit import default_timer as timer
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from gcnn.graph import *
from gcnn.coarsening import graclus

In [ ]:
gr_size = 28
gr = grid_coordinates(gr_size)
gr.shape

In [ ]:
def knn(z, k=4, metric='euclidean'):
    d = spatial.distance.pdist(z, metric)
    w = spatial.distance.squareform(d)
    
    knc = KNeighborsClassifier(n_neighbors=k, metric=metric)
    knc.fit(z, np.zeros_like(z)) 
    
    closest = knc.kneighbors(z, return_distance=False)

    w = np.zeros_like(w)
    for i in range(len(w)):
        w[i, closest[i]] = 1
        w[i, i] = 0
    
    return w

In [ ]:
mask = knn(gr, k=8) > 0
plt.spy(mask[:40, :40]);

In [ ]:
graphs, perm = graclus.coarsen(sp.sparse.csr.csr_matrix(mask), levels=3, self_connections=False)

In [ ]:
laps = [sp.sparse.csgraph.laplacian(g, normed=True) for g in graphs[:-1]]

In [ ]:
n = laps[0].shape[0]
n

In [ ]:
def fourier(laplacian):
    eigenvalues, eigenvectors = sp.linalg.eigh(laplacian)
    return eigenvectors

In [ ]:
l0 = torch.from_numpy(laps[0].todense()).float()
l0.size()

In [ ]:
l1 = torch.from_numpy(laps[1].todense()).float()
l1.size()

In [ ]:
l2 = torch.from_numpy(laps[2].todense()).float()
l2.size()

In [ ]:
f0 = torch.from_numpy(fourier(laps[0].todense())).float()
f0.size()

In [ ]:
f1 = torch.from_numpy(fourier(laps[1].todense())).float()
f1.size()

In [ ]:
f2 = torch.from_numpy(fourier(laps[2].todense())).float()
f2.size()

In [ ]:
data_folder = './data'
train = dset.MNIST(data_folder, train=True, download=True)
test = dset.MNIST(data_folder, train=False)

In [ ]:
train_x, train_y = zip(*train)
test_x, test_y = zip(*test)

train_x = np.stack([np.r_[np.reshape(i, -1) / 255.0, np.zeros(n - gr_size * gr_size)] for i in train_x])[:, perm]
test_x = np.stack([np.r_[np.reshape(i, -1) / 255.0, np.zeros(n - gr_size * gr_size)] for i in test_x])[:, perm]

train_x = torch.from_numpy(train_x).float()
test_x = torch.from_numpy(test_x).float()
train_y = torch.from_numpy(np.array(train_y)).long()
test_y = torch.from_numpy(np.array(test_y)).long()

train_x.shape, train_y.shape

In [ ]:
test_x.shape, test_y.shape

In [ ]:
cuda = torch.cuda.is_available()
cuda

In [ ]:
batch_size = 100
train_loader = data_utils.DataLoader(data_utils.TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(test_x, test_y), batch_size=batch_size, shuffle=True)

In [ ]:
from gcnn.nets import *

In [ ]:
net = PaperGCFC(l0.cuda() if cuda else l0, l2.cuda() if cuda else l2)

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(), lr=0.0003)#, weight_decay=0.005)
optimizer = torch.optim.SGD(net.parameters(), lr=0.02, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

writer = SummaryWriter()

if cuda:
    net.cuda()
    criterion.cuda()

#writer.add_graph(net, net(Variable(train_x[0].unsqueeze(0)).cuda()))
    
epoch_train_loss = []
epoch_test_loss = []

net

In [ ]:
def loop(net, loader, training=False):
    
    running_loss = 0
    targets = []
    predictions = []
    start = timer()
    
    for batch_id, (x, y) in enumerate(loader):
        x = Variable(x)
        y = Variable(y)
        
        if cuda:
            x = x.cuda()
            y = y.cuda()
        
        optimizer.zero_grad()
        
        outputs = net(x)
        loss = criterion(outputs, y)
        
        ps = torch.cat([p.view(-1) for p in net.fc.parameters()])
        #loss += 0.0005 * F.l1_loss(ps, target=torch.zeros_like(ps), size_average=False)
        loss += 5e-4 * F.mse_loss(ps, target=torch.zeros_like(ps), size_average=False)


        if training:
            loss.backward()
            optimizer.step()

        running_loss += loss.data.cpu()[0]
        targets.extend(y.data.cpu().numpy())
        predictions.extend(outputs.data.cpu().numpy())
        
        if (batch_id + 1) % 10 == 0 and training:
            print(running_loss / (batch_id * batch_size), end='\r')
        
    if training:
        scheduler.step()
        
    return np.array(targets), np.array(predictions), running_loss, (timer() - start)

In [ ]:
for e in range(5):

    train_targets, train_preds, train_loss, train_duration = loop(net, train_loader, training=True)
    writer.add_scalar('data/train_loss', train_loss, e)
    train_acc = accuracy_score(train_targets, train_preds.argmax(axis=1))
    writer.add_scalar('data/train_accuracy', train_acc, e)
      
    test_targets, test_preds, test_loss, test_duration = loop(net, test_loader, training=False)
    writer.add_scalar('data/test_loss', test_loss, e)
    test_acc = accuracy_score(test_targets, test_preds.argmax(axis=1))
    writer.add_scalar('data/test_accuracy', test_acc, e)

    train_loss /= len(train)
    test_loss /= len(test)
    
    epoch_train_loss.append(train_loss)
    epoch_test_loss.append(test_loss)
    
    print(e, 'Training {:.4f} {:.2f}% Testing {:.4f} {:.2f}% Duration {:.2f}s {:.2f}s'.format(
        train_loss, train_acc * 100, test_loss, test_acc * 100, train_duration, test_duration
    ))

In [ ]:
plt.plot(epoch_train_loss, label='train')
plt.plot(epoch_test_loss, label='test')
plt.legend();

In [ ]:
train_targets, train_preds, train_loss, train_duration = loop(net, train_loader)
accuracy_score(train_targets, train_preds.argmax(axis=1)), train_duration

In [ ]:
np.count_nonzero(train_preds.argmax(axis=1) == train_targets) / 60000

In [ ]:
train_preds.argmax(axis=1)

In [ ]:
test_targets, test_preds, test_loss, test_duration = loop(net, test_loader)
accuracy_score(test_targets, test_preds.argmax(axis=1)), test_duration